### Import libraries and create DataFrame from csv 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("RB_Data.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,ATT,YDS,Y/A,LG,20+,TD,REC,TGT,YDS.1,Y/R,TD.1,G,FPTS,FPTS/G
0,1,Christian McCaffrey (SF),272,1459,5.4,72,9,14,67,83,564,8.4,7,16,391.3,24.5
1,2,Breece Hall (NYJ),223,994,4.5,83,7,5,76,95,591,7.8,4,17,290.5,17.1
2,3,Travis Etienne Jr. (JAC),267,1008,3.8,62,6,11,58,73,476,8.2,1,17,282.4,16.6
3,4,Rachaad White (TB),272,990,3.6,38,3,6,64,70,549,8.6,3,17,267.9,15.8
4,5,Raheem Mostert (MIA),209,1012,4.8,49,7,18,25,32,175,7.0,3,15,267.7,17.8
5,6,Joe Mixon (HOU),257,1034,4.0,44,3,9,52,64,376,7.2,3,17,267.0,15.7
6,7,Kyren Williams (LAR),228,1144,5.0,56,7,12,32,48,206,6.4,3,12,255.0,21.3
7,8,Derrick Henry (BAL),280,1167,4.2,69,5,12,28,36,214,7.6,0,17,246.7,14.5
8,9,Bijan Robinson (ATL),214,976,4.6,38,7,4,58,86,487,8.4,4,17,246.3,14.5
9,10,Jahmyr Gibbs (DET),182,945,5.2,36,10,10,52,71,316,6.1,1,15,242.1,16.1


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [3]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/A', 'LG', 'Y/R', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/A,LG,Y/R,G,FPTS,FPTS/G,ATT/game,YDS/game,20+/game,TD/game,REC/game,TGT/game,YDS.1/game,TD.1/game
0,1,Christian McCaffrey (SF),5.4,72,8.4,16,391.3,24.5,17.0,91.2,0.6,0.9,4.2,5.2,35.2,0.4
1,2,Breece Hall (NYJ),4.5,83,7.8,17,290.5,17.1,13.1,58.5,0.4,0.3,4.5,5.6,34.8,0.2
2,3,Travis Etienne Jr. (JAC),3.8,62,8.2,17,282.4,16.6,15.7,59.3,0.4,0.6,3.4,4.3,28.0,0.1
3,4,Rachaad White (TB),3.6,38,8.6,17,267.9,15.8,16.0,58.2,0.2,0.4,3.8,4.1,32.3,0.2
4,5,Raheem Mostert (MIA),4.8,49,7.0,15,267.7,17.8,13.9,67.5,0.5,1.2,1.7,2.1,11.7,0.2
5,6,Joe Mixon (HOU),4.0,44,7.2,17,267.0,15.7,15.1,60.8,0.2,0.5,3.1,3.8,22.1,0.2
6,7,Kyren Williams (LAR),5.0,56,6.4,12,255.0,21.3,19.0,95.3,0.6,1.0,2.7,4.0,17.2,0.2
7,8,Derrick Henry (BAL),4.2,69,7.6,17,246.7,14.5,16.5,68.6,0.3,0.7,1.6,2.1,12.6,0.0
8,9,Bijan Robinson (ATL),4.6,38,8.4,17,246.3,14.5,12.6,57.4,0.4,0.2,3.4,5.1,28.6,0.2
9,10,Jahmyr Gibbs (DET),5.2,36,6.1,15,242.1,16.1,12.1,63.0,0.7,0.7,3.5,4.7,21.1,0.1


### Calculate the correlations for the final stats across different conditions

In [4]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/A,0.66,0.34,0.30,0.32,0.40
LG,0.85,0.76,0.57,0.26,0.61
Y/R,0.59,0.22,0.01,-0.18,0.16
ATT/game,0.94,0.92,0.79,0.46,0.78
YDS/game,0.95,0.95,0.89,0.73,0.88
20+/game,0.77,0.75,0.65,0.24,0.60
TD/game,0.84,0.83,0.74,0.53,0.73
REC/game,0.87,0.85,0.53,0.45,0.67
TGT/game,0.87,0.85,0.56,0.48,0.69
YDS.1/game,0.87,0.84,0.53,0.45,0.67


### Assign the weights for the final stats

In [5]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/A,0.40,0.16,0.07
LG,0.61,0.37,0.08
Y/R,0.16,0.03,0.06
ATT/game,0.78,0.60,0.13
YDS/game,0.88,0.78,0.15
20+/game,0.60,0.36,0.08
TD/game,0.73,0.54,0.12
REC/game,0.67,0.46,0.08
TGT/game,0.69,0.48,0.08
YDS.1/game,0.67,0.45,0.08


### Multiply the assigned weights to the final stats and calculate the score

In [6]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,Y/A_weighted,LG_weighted,ATT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted,REC/game_weighted,TGT/game_weighted,YDS.1/game_weighted,TD.1/game_weighted
0,1,Christian McCaffrey (SF),24.5,0.4,5.6,2.1,13.3,0.0,0.1,0.3,0.4,2.9,0.0
1,2,Breece Hall (NYJ),17.1,0.3,6.5,1.7,8.5,0.0,0.0,0.4,0.5,2.9,0.0
2,3,Travis Etienne Jr. (JAC),16.6,0.3,4.9,2.0,8.6,0.0,0.1,0.3,0.4,2.3,0.0
3,4,Rachaad White (TB),15.8,0.2,3.0,2.0,8.5,0.0,0.0,0.3,0.3,2.7,0.0
4,5,Raheem Mostert (MIA),17.8,0.3,3.8,1.8,9.8,0.0,0.1,0.1,0.2,1.0,0.0
5,6,Joe Mixon (HOU),15.7,0.3,3.4,1.9,8.9,0.0,0.1,0.3,0.3,1.8,0.0
6,7,Kyren Williams (LAR),21.3,0.3,4.4,2.4,13.9,0.0,0.1,0.2,0.3,1.4,0.0
7,8,Derrick Henry (BAL),14.5,0.3,5.4,2.1,10.0,0.0,0.1,0.1,0.2,1.0,0.0
8,9,Bijan Robinson (ATL),14.5,0.3,3.0,1.6,8.4,0.0,0.0,0.3,0.4,2.4,0.0
9,10,Jahmyr Gibbs (DET),16.1,0.3,2.8,1.5,9.2,0.1,0.1,0.3,0.4,1.7,0.0


### Model training

In [7]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.4665372993238061
Test Set MSE: 0.35268943971249417
Random Forest Test Set MSE: 0.06359122623833528


/tmp/ipykernel_2519/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [8]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("RB_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,Y/A,G,FPTS,FPTS/G,YDS/game,20+/game,TD/game,REC/game,TGT/game
Rank,,,,,,,,,,,,,
1,1,Christian McCaffrey (SF),10.00,0,5.4,16,391.3,24.5,91.2,0.6,0.9,4.2,5.2
7,2,Kyren Williams (LAR),7.82,5,5.0,12,255.0,21.3,95.3,0.6,1.0,2.7,4.0
24,3,De'Von Achane (MIA),7.53,21,7.8,11,190.7,17.3,72.7,0.7,0.7,2.5,3.4
2,4,Breece Hall (NYJ),7.51,-2,4.5,17,290.5,17.1,58.5,0.4,0.3,4.5,5.6
5,5,Raheem Mostert (MIA),7.22,0,4.8,15,267.7,17.8,67.5,0.5,1.2,1.7,2.1
3,6,Travis Etienne Jr. (JAC),7.14,-3,3.8,17,282.4,16.6,59.3,0.4,0.6,3.4,4.3
11,7,Alvin Kamara (NO),7.00,4,3.9,13,233.0,17.9,53.4,0.0,0.4,5.8,6.6
18,8,James Conner (ARI),6.89,10,5.0,13,201.5,15.5,80.0,0.8,0.5,2.1,2.5
33,9,Jonathan Taylor (IND),6.87,24,4.4,10,156.4,15.6,74.1,0.4,0.7,1.9,2.3
